In [88]:
# ✅ STEP 1: Install Required Libraries
!pip install -q transformers PyPDF2

In [89]:
# ✅ STEP 2: Upload Resume and Job Description Files
from google.colab import files
import PyPDF2

print("📄 Upload your RESUME (PDF or TXT)")
resume_file = files.upload()

print("📄 Upload JOB DESCRIPTION (PDF or TXT)")
job_file = files.upload()

📄 Upload your RESUME (PDF or TXT)


Saving Kshitiz Mathur.pdf to Kshitiz Mathur (8).pdf
📄 Upload JOB DESCRIPTION (PDF or TXT)


Saving Job description.txt to Job description (7).txt


In [90]:
# ✅ STEP 3: Extract Text from Files
def read_file(file_dict):
    filename = list(file_dict.keys())[0]
    if filename.endswith(".pdf"):
        reader = PyPDF2.PdfReader(filename)
        return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    else:
        with open(filename, "r", encoding='utf-8') as f:
            return f.read()

resume_text = read_file(resume_file)
job_text = read_file(job_file)

In [96]:
# ✅ Step 4 (updated): Summarize resume AND job description
from transformers import pipeline

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Truncate to first 1000 chars (safe input length) before summarizing
resume_summary = summarizer(resume_text[:1000], max_length=100, min_length=50, do_sample=False)[0]['summary_text']
job_summary = summarizer(job_text[:1000], max_length=100, min_length=50, do_sample=False)[0]['summary_text']


Device set to use cpu


In [108]:
# ✅ Step 5 (unchanged): Load FLAN-T5 for Generative QA
qa = pipeline("text2text-generation", model="google/flan-t5-large")


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [111]:
few_shot_example = """
Resume:
Alex Johnson is a machine learning engineer with expertise in Python, scikit-learn, and Pandas. He has experience deploying models via Flask and analyzing time series data.

Job Description:
We're hiring an ML Engineer with experience in TensorFlow, AWS, NLP, and MLOps tools like MLflow.

Question: What skills in the resume match the job description?
Answer: Python, machine learning, scikit-learn, model deployment with Flask.

Question: Question: What qualifications are missing in the resume?
Answer: TensorFlow, AWS/cloud platforms, NLP techniques, and MLOps tools such as MLflow.

Question: What improvements can be made to align the resume with the job?
Answer: Add experience with TensorFlow, cloud deployment (AWS), and include any projects involving NLP or MLflow. Highlight collaboration or agile workflow experience if available.
"""


In [112]:
# ✅ Step 7: Final QA loop with compact summaries
questions = [
    "What skills in the resume match the job description?",
    "What qualifications are missing in the resume?",
    "What improvements can be made to align the resume with the job?"
]

for q in questions:
    task_instruction = task_templates[q]

    prompt = f"""
You are a resume evaluation assistant.

{few_shot_example.strip()}

### TASK
{task_instruction}

### RESUME
{resume_summary}

### JOB DESCRIPTION
{job_summary}

### QUESTION
{q}

### ANSWER
""".strip()

    print(f"\n🧠 Question: {q}")
    result = qa(prompt, max_new_tokens=256, do_sample=False)
    print("📌 Answer:", result[0]['generated_text'].strip())



🧠 Question: What skills in the resume match the job description?
📌 Answer: Python, machine learning, scikit-learn, TensorFlow, PyTorch

🧠 Question: What qualifications are missing in the resume?
📌 Answer: Technical qualifications, certifications, or tools that are expected in the job description but not present in the resume

🧠 Question: What improvements can be made to align the resume with the job?
📌 Answer: Include any projects involving NLP or MLflow. Highlight collaboration or agile workflow experience if available.
